In [2]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import Generate_word2vec_features
import os
import sys
import RunModels
sys.path.append( os.path.join(".."))
from UtilityFunctions import CommonHelpers,PreprocessHelpers,FeatureEngineering
import numpy as np
import pandas as pd
import re

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sharath/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sharath/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sharath/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# Constants
book_pickle_path="../data/preprocessed/books_english.pickle"
name_pickle_path = "../data/preprocessed/names_english.pickle"

In [7]:
def get_number_id(file_name):
    regex = r"([a-z_A-Z]+)(\d+)"
    match = re.search(regex, file_name)
    if match!=None:
        return match.group(2)

In [8]:
list_of_words = CommonHelpers.load_pickle(book_pickle_path)
book_names =  CommonHelpers.load_pickle(name_pickle_path)

In [9]:
df = pd.read_csv("../data/pg_catalog.csv")
df["category"] = df["LoCC"].str[:1]
category_list_train = [int(get_number_id(each)) for each in book_names]
selected_books = df[df["Text#"].isin(category_list_train)]
categories=[]
titles=[]
for each in category_list_train:
    categories.append(selected_books[selected_books["Text#"]==each]["category"].values[0])
    titles.append(selected_books[selected_books["Text#"]==each]["Title"].values[0])

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [1]:
# For testing
"""
number = 420
print(list_of_words[number][:100])
print(">>>>>>>>>>>>>>>.")
print(book_names[number],categories[number],titles[number],category_list_train[number])
"""

'\nnumber = 420\nprint(list_of_words[number][:100])\nprint(">>>>>>>>>>>>>>>.")\nprint(book_names[number],categories[number],titles[number],category_list_train[number])\n'

In [10]:
# Google Word2Vec book vectors
gen = Generate_word2vec_features.Word2Vec_Features()
gen.load_data(cache=True)
train_vec =np.nan_to_num( gen.book_vectors)

In [ ]:
# TF-IDF
# load the final vectors of size 1400*X (vary X ) into variable train_vec

In [ ]:
# Custom Word2Vec
# load the final vectors of size 1400*X (vary X ) into variable train_vec

In [12]:
split_obj = RunModels.Split_Data(train_vec,categories)
split_obj.get_split()

In [16]:
names = [
    "Nearest Neighbors",
    "RBF SVM",
    "Random Forest",
    "Neural Net",
    "Naive Bayes",
]
classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    RandomForestClassifier( n_estimators=400),
    MLPClassifier(alpha=0.05, max_iter=10000),
    GaussianNB(),
]

In [15]:
for name, clf in zip(names, classifiers):
    model = RunModels.Run_Model(clf,split_obj)
    model.run_model()
    result = model.get_predictions()
    matrix,accuracy = model.get_metrics()
    print(name,accuracy)

Nearest Neighbors 0.5428571428571428
RBF SVM 0.5821428571428572
RF esti = 1000 0.6357142857142857
RF max depth =15  0.6285714285714286
RF no max features 0.625
rf just estimators 0.6571428571428571
Neural Net 0.6678571428571428
Naive Bayes 0.5035714285714286
